In [1]:
import os
import sys

root_path = os.path.abspath(os.path.join(os.getcwd(), '../..'))
sys.path.append(root_path)

In [2]:
from core.backtesting.optimizer import BacktestingConfig, BaseStrategyConfigGenerator
from hummingbot.strategy_v2.executors.position_executor.data_types import TrailingStop
from controllers.directional_trading.macd_bb_v1 import MACDBBV1ControllerConfig
from decimal import Decimal


class MACDBBConfigGenerator(BaseStrategyConfigGenerator):
    """
    Strategy configuration generator for MACD and Bollinger Bands optimization.
    """
    def generate_config(self, trial) -> BacktestingConfig:
        # Suggest hyperparameters using the trial object
        fast_ma = trial.suggest_int("fast_ma", 9, 59, step=10)
        slow_ma = trial.suggest_int("slow_ma", 21, 201, step=10)
        signal_ma = trial.suggest_int("signal_ma", 10, 200, step=10)
        bb_length = trial.suggest_int("bb_length", 10, 200, step=10)
        bb_std = trial.suggest_float("bb_std", 1.0, 2.5, step=0.1)
        total_amount_quote = 1000
        max_executors_per_side = trial.suggest_int("max_executors_per_side", 1, 10)
        take_profit = trial.suggest_float("take_profit", 0.01, 0.1, step=0.01)
        stop_loss = trial.suggest_float("stop_loss", 0.01, 0.1, step=0.01)
        trailing_stop_activation_price = trial.suggest_float("trailing_stop_activation_price", 0.004, 0.02, step=0.001)
        trailing_delta_ratio = trial.suggest_float("trailing_delta_ratio", 0.05, 0.1, step=0.01)
        trailing_stop_trailing_delta = trailing_stop_activation_price * trailing_delta_ratio
        time_limit = 60 * 60 * 24 * 2
        cooldown_time = 60 * 15

        # Create the strategy configuration
        config = MACDBBV1ControllerConfig(
            connector_name="binance_perpetual",
            trading_pair="1000PEPE-USDT",
            macd_fast=fast_ma,
            macd_slow=slow_ma,
            macd_signal=signal_ma,
            bb_length=bb_length,
            bb_std=bb_std,
            total_amount_quote=Decimal(total_amount_quote),
            take_profit=Decimal(take_profit),
            stop_loss=Decimal(stop_loss),
            trailing_stop=TrailingStop(
                activation_price=Decimal(trailing_stop_activation_price),
                trailing_delta=Decimal(trailing_stop_trailing_delta),
            ),
            time_limit=time_limit,
            max_executors_per_side=max_executors_per_side,
            cooldown_time=cooldown_time,
        )

        # Return the configuration encapsulated in BacktestingConfig
        return BacktestingConfig(config=config, start=self.start, end=self.end)

In [3]:
from core.backtesting.optimizer import StrategyOptimizer
import datetime

start_date = datetime.datetime(2024, 8, 1)
end_date = datetime.datetime(2024, 8, 31)
config_generator = MACDBBConfigGenerator(start_date=start_date, end_date=end_date)

optimizer = StrategyOptimizer(root_path=root_path)
await optimizer.optimize(
    study_name="macd_bb_v1_with_optimizer",
    config_generator=config_generator,
    n_trials=100,
)

[I 2024-09-12 07:09:43,278] Using an existing study with name 'macd_bb_v1_with_optimizer' instead of creating a new one.
2024-09-12 07:09:44,700 - asyncio - ERROR - Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x1425fa260>
2024-09-12 07:09:44,703 - asyncio - ERROR - Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x140a55c60>, 267794.876828583)]']
connector: <aiohttp.connector.TCPConnector object at 0x1425fa170>
2024-09-12 07:09:47,270 - asyncio - ERROR - Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x1425ebc70>
2024-09-12 07:09:47,271 - asyncio - ERROR - Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x1420fbf40>, 267797.434463333)]']
connector: <aiohttp.connector.TCPConnector object at 0x1425eba90>
2024-09-12 07:09:57,926 - asyncio - ERROR - Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x1425ebf70>
2024-09-12

In [4]:
optimizer.launch_optuna_dashboard()